Train ocr with keras

dataseti githubdan çek ve unziple

In [1]:
!wget -q https://github.com/onuralpArsln/toJava/raw/refs/heads/main/PayDayProject/dataset.zip

In [ ]:
!unzip -q dataset.zip

Train için idinrimeler ve importları yap

In [2]:
!pip -q install tensorflow opencv-python matplotlib
